In [ ]:
import pandas as pd
import numpy as np
import math
import plotly.express as px
import plotly.graph_objects as go
#! pip install kaleido
import kaleido

In [123]:
from plotly.subplots import make_subplots

directory = '/Users/gillislowry/Documents/Kaltenegger/'
filename = 'VisalizationOrbit.csv'
df = pd.read_csv(directory + filename)

def graph(df):
    
    for i, row in df.iterrows(): 

        
        print(row['HostStarName'])

        # Constants
        G = 6.67430e-11  # Gravitational constant  

        #Create gradient for star color based on stellar temp
        scale = [(0.0,"#522020"),(0.07,"#800000"),(0.09,"#b80000"),(0.13,"#ede968"),(0.155,"#e8e8e8"),(0.3,"#97dadb"),(1.0,"#757aff")]
        backgroundcolor = "rgb(0,2,38)"

        T = {}
        T_list = []
        orbs = []
        orbs_au = []
        eccs = []
        color = float(row['Temp']) 

        # Find maximum time to complete an orbit (this will be half of the total animation runtime)
        for k in range(0,8):
            
            if row['Orb_in_meters'+str(k)] != 0:
                print(row['Orb_in_meters'+str(k)])
            if row['OrbPer'+str(k)] != 0:
                print(row['OrbPer'+str(k)])
            if row['Name'+str(k)] != 0:
                print(row['Name'+str(k)])

            if row['Orb'+str(k)] != 0.0 and row['Name'+str(k)] != 0: 
                T[row['Name'+str(k)]] = ( 2 * np.pi * np.sqrt(row['Orb_in_meters'+str(k)]**3 / (G * row['StarMass'])))
                T_list.append( 2 * np.pi * np.sqrt(row['Orb_in_meters'+str(k)]**3 / (G * row['StarMass'])))
                orbs.append(row['Orb_in_meters'+str(k)])
                orbs_au.append(row['Orb'+str(k)])
                eccs.append(row['Ecc'+str(k)])
                
                    
            elif row['Orb'+str(k)] == 0.0 and row['Name'+str(k)] != 0 and row['OrbPer'+str(k)] != 0:
                T[row['Name'+str(k)]] = row['OrbPer'+str(k)]*24*60*60
                T_list.append(row['OrbPer'+str(k)]*24*60*60)
                orbs.append(((G * row['StarMass']*(row['OrbPer'+str(k)]*24*60*60)**2) / (4*np.pi**2))**(1/3))
                orbs_au.append((((G * row['StarMass']*(row['OrbPer'+str(k)]*24*60*60)**2) / (4*np.pi**2))**(1/3))/1.496e11)
                eccs.append(row['Ecc'+str(k)])
                    
            elif row['Name'+str(k)] != 0:
                print("bad")
                T[row['Name'+str(k)]] = 0
                T_list.append(0)
                orbs.append(np.nan)
                orbs_au.append(np.nan)
                eccs.append(np.nan)
            
            if len(eccs) !=0 and np.max(eccs) > 0.5:
                ecc_adjust = 1.2
                ecc_adjust2 = 1.3
                if np.max(eccs) > 0.8:
                    ecc_adjust = 1.4
                    ecc_adjust2 = 1.6
            else:
                ecc_adjust2 = 0
                ecc_adjust = 1

        #print(T_list)    
                  
        if np.max(T_list) == 0:
            noData = True
        else:
            noData = False
            #print(str(row['Name'+str(k)]) + " " + str(T[row['Name'+str(k)]]))


        # Create time array
        if noData != True:
            if np.min(orbs_au) > 2 and np.min(orbs_au) <= 100:
                frames = 612
                t = np.linspace(0, np.max(T_list), frames)
                annotation_divide = 612
            elif np.min(orbs_au) > 100:
                frames = 1200
                t = np.linspace(0, np.max(T_list), frames)
                annotation_divide = 1200
            else:
                frames = 612
                t = np.linspace(0, 2*np.max(T_list), frames)
                annotation_divide = 306
        else:
            frames = 1
            t = np.linspace(0, 1, frames)
            annotation_divide = 1

        # Determine whether time annotation should be in hours, days, or years:
        if noData != True and np.max(T_list) <= 100000:
            unit = " hours"
        elif noData != True and np.max(T_list) <= 50000000:
            unit = " days"
        elif noData != True:
            unit = " years"
        else:
            unit = ''

        # Create empty frame list:
        frames_data = []

        for j in range(0, len(t)): # For each frame,

                #print("Frame " + str(j))
                planet = []

                #Determine time to be annotated on each frame:
                if noData != True and np.max(T_list) <= 100000:
                    time = np.round(((np.max(T_list)/annotation_divide)*j)/60/60,3)
                elif noData != True and np.max(T_list) <= 50000000: 
                    time = np.round(((np.max(T_list)/annotation_divide)*j)/60/60/24,3)
                elif noData != True:
                    time = np.round(((np.max(T_list)/annotation_divide)*j)/60/60/24/365.256363004,3)
                else:
                    time = ''

                for k in range(0, len(T)): # For each planet,

                    if row['Name'+str(k)] != 0 and noData != True:

                        #theta = 2 * np.pi * t / T[row['Name'+str(k)]]
                        #r = row['Orb_in_meters'+str(k)] * (1 - row['Ecc'+str(k)] ** 2) / (1 + row['Ecc'+str(k)] * np.cos(2 * np.pi * t / T[row['Name'+str(k)]]))
                        # x = (r * np.cos(theta))
                        # y = (r * np.sin(theta))
                        x = list((row['Orb_in_meters'+str(k)] * (1 - float(row['Ecc'+str(k)]) ** 2) / (1 + float(row['Ecc'+str(k)]) * np.cos(2 * np.pi * t / T[row['Name'+str(k)]])) * np.cos(2 * np.pi * t / T[row['Name'+str(k)]]))+2*(row['Orb_in_meters'+str(k)]*float(row['Ecc'+str(k)])))
                        y = list(row['Orb_in_meters'+str(k)] * (1 - float(row['Ecc'+str(k)]) ** 2) / (1 + float(row['Ecc'+str(k)]) * np.cos(2 * np.pi * t / T[row['Name'+str(k)]])) * np.sin(2 * np.pi * t / T[row['Name'+str(k)]]))
                        # Non-eccentricity version:
                        # x = list(row['Orb_in_meters'+str(k)] * np.cos(2 * np.pi * t / T[row['Name'+str(k)]]))
                        # y = list(row['Orb_in_meters'+str(k)] * np.sin(2 * np.pi * t / T[row['Name'+str(k)]]))
                        orbdist = orbs_au[k]

                        data_by_planet = {
                        "x": [x[j]],
                        "y": [y[j]],
                        "mode": "markers+text",
                        "marker": {
                            "size": (5 + row['Size'+str(k)]),
                            "color": "rgb(236, 85, 250)"
                        },
                        "name": (row['HostStarName']+ ' ' + row['Name'+str(k)]),
                        "customdata": [orbdist],
                        "hovertemplate": "<i>Orbital distance</i>: %{customdata} AU",
                        "textposition": 'top center'
                        }

                        graph_one_planet = go.Scatter(data_by_planet)
                        graph_one_orbit = go.Scatter(
                            x=x, y=y, mode='markers', hoverinfo='skip', marker=dict(size=1, color='white')
                            )

                        # Add each planet's position to planet list
                        planet.append(graph_one_planet)
                        planet.append(graph_one_orbit)

                if noData != True and np.max(T_list) > 0.0:

                        # Add another annotation if hab zone could not be calculated
                        if row['RecentVenus'] == 0 or row['MaximumGreenhouse'] == 0: 
                            frame_layout = go.Layout(annotations=
                                    [go.layout.Annotation(text="Time elapsed: " + str(time) + unit + "<br>Unable to calculate habitable zone for " + str(row['HostStarName']),
                                      xref="paper", yref="paper",
                                        font = dict(color = 'white'),
                                      x=0.92, y=0.9, showarrow=False)],
                                    )  
                        else:
                            frame_layout = go.Layout(annotations=
                                    [go.layout.Annotation(text="Time elapsed: " + str(time) + unit,
                                      xref="paper", yref="paper",
                                        font = dict(color = 'white'),
                                      x=0.92, y=0.9, showarrow=False)],
                                    )      
                else:
                            frame_layout = go.Layout(annotations=
                                [go.layout.Annotation(text="No orbit data for " + str(row['HostStarName']),
                              xref="paper", yref="paper",
                                font = dict(color = 'white', size = 15),
                              x=0.5, y=0.55, showarrow=False)],)


                # Graph star for each frame:
                graph_star = go.Scatter(
                        x=[0,0,0], y=[0,0,0], mode='markers', marker=dict(color = [400,40000,color], coloraxis='coloraxis', size=10),
                        name=row['HostStarName'],
                        hoverinfo='skip'
                        )

                planet.append(graph_star)

                # Add list of planet positions to the frame for each image
                frame = (go.Frame(data=planet,layout=frame_layout))

                # Append one frame 
                frames_data.append(frame)


        # Create plot
        fig = go.Figure(data=[go.Scatter()
                              for traces in planet])

        # Set frames attribute
        fig.frames = frames_data
        for k in range(len(fig.frames)):
            fig.frames[k]['layout'].update()


        #Graph hab zone boundaries for each frame:  

        fig.add_shape(type="circle",
            xref="x", yref="y",
            fillcolor="rgba(110, 216, 255, .5)",
            x0=(-1)*row['EarlyMars']*1.496e+11, y0=(-1)*row['EarlyMars']*1.496e+11, x1=row['EarlyMars']*1.496e+11, y1=row['EarlyMars']*1.496e+11,
            line = dict(width=0),
            layer = "below"
            )
        fig.add_shape(type="circle",
            xref="x", yref="y",
            fillcolor="rgba(129, 255, 112, .2)",
            x0=(-1)*row['MaximumGreenhouse']*1.496e+11, y0=(-1)*row['MaximumGreenhouse']*1.496e+11, x1=row['MaximumGreenhouse']*1.496e+11, y1=row['MaximumGreenhouse']*1.496e+11,
            line = dict(width=0),
            layer = "below"
            )
        fig.add_shape(type="circle",
            xref="x", yref="y",
            fillcolor="rgba(252, 220, 101, .5)",
            x0=(-1)*row['RunawayGreenhouse']*1.496e+11, y0=(-1)*row['RunawayGreenhouse']*1.496e+11, x1=row['RunawayGreenhouse']*1.496e+11, y1=row['RunawayGreenhouse']*1.496e+11,
            line = dict(width=0),
            layer = "below"
            )
        fig.add_shape(type="circle",
            xref="x", yref="y",
            fillcolor=backgroundcolor,
            x0=(-1)*row['RecentVenus']*1.496e+11, y0=(-1)*row['RecentVenus']*1.496e+11, x1=row['RecentVenus']*1.496e+11, y1=row['RecentVenus']*1.496e+11,
            layer = "below",
            line = dict(width=0)
            )


        # Update layout
        
        if noData != True: 
            range_x = [-1.551 * np.max(orbs)*ecc_adjust+ecc_adjust2*np.max(orbs), 1.551 * np.max(orbs)*ecc_adjust+ecc_adjust2*np.max(orbs)]
            range_y = [-1.11 * np.max(orbs)*ecc_adjust, 1.11 * np.max(orbs)*ecc_adjust]
        else:
            range_x = [-1,1]
            range_y = [-1,1]
            
        if noData != True: 
            minrange_x = [-1.551 * np.min(orbs)*ecc_adjust, 1.551 * np.min(orbs)*ecc_adjust]
            minrange_y = [-1.11 * np.min(orbs)*ecc_adjust, 1.11 * np.min(orbs)*ecc_adjust]
        else:
            minrange_x = [-1,1]
            minrange_y = [-1,1]
            
        if row['EarlyMars'] != 0:
            habrange_x = [-1.551 * row['EarlyMars']*1.496e+11, 1.551 * row['EarlyMars']*1.496e+11]
            habrange_y = [-1.11 * row['EarlyMars']*1.496e+11, 1.11 * row['EarlyMars']*1.496e+11]
        else:
            habrange_x = [-1,1]
            habrange_y = [-1,1]
            
        fig.update_layout(
            xaxis=dict(
                range=range_x,
                showgrid=False,
                zeroline=False,
                showline=False,
                showticklabels=False
            ),
            yaxis=dict(
                range=range_y,
                showgrid=False,
                zeroline=False,
                showline=False,
                showticklabels=False
            ),
            coloraxis_showscale=False,
            coloraxis=dict(colorscale = scale),
            plot_bgcolor=backgroundcolor, 
            paper_bgcolor=backgroundcolor,
            showlegend=False,
            margin = dict(l = 0, r = 0, b = 0, t = 0),
            width=1400, 
            height=1000, 
            updatemenus=[{
                'buttons': [
                                    {
                        'args': [[None], {'frame': {'duration': 0, 'redraw': True}, 'mode': 'immediate', 'transition': {'duration': 0}}],
                        'label': 'Pause/Stop',
                        'method': 'animate'
                    },
                    {
                        'args': [None, {'frame': {'duration': 0, 'redraw': False}, 'transition': {'duration': 800}, 'fromcurrent': True}],
                        'label': 'Play',
                        'method': 'animate'
                    },
                    {
                        'args': [None, {'frame': {'duration': 0, 'redraw': True}, 'transition': {'duration': 800}, 'fromcurrent': True}],
                        'label': 'Play with time counter',
                        'method': 'animate'
                    },
                    {
                        'args': [{'xaxis.range':habrange_x,
                                 'yaxis.range':habrange_y}],
                        'label': 'Zoom to habitable zone',
                        'method': 'relayout'
                    },
                    {
                        'args': [{'xaxis.range':minrange_x,
                                 'yaxis.range':minrange_y}],
                        'label': 'Zoom to closest orbit',
                        'method': 'relayout'
                    },
                    {
                        'args': [{'xaxis.range':range_x,
                                 'yaxis.range':range_y}],
                        'label': 'Reset Zoom',
                        'method': 'relayout'
                    }
                ],
                'direction': 'left',
                'showactive': True,
                'type': 'buttons',
                'x': 0.5,
                'xanchor': 'center',
                'y': 1.0,
                'yanchor': 'top',
                'bgcolor': 'white', 
                'font': {'color':'black', 'family':'avenir'}
            }]
            )



        print(row['HostStarName'] + " figure 1 done!")
        fig.show()


        if len(T_list) <= 2 or noData == True: 
            showtextOnAutoplay = True
        else:
            showtextOnAutoplay = False

        fig.write_html(directory+ "/orbit animations/"+row['HostStarName']+".html", 
                       auto_play = True, animation_opts = dict(frame = dict(duration = 0, redraw = showtextOnAutoplay), 
                        transition = dict(duration = 800), fromcurrent = True),
                      config = {'scrollZoom': True, 'displayModeBar': True, 
                                'modeBarButtonsToRemove': ['autoScale2d', 'select2d', 'lasso2d']})
        #'autosizable': True, 'fillFrame': True
        

In [ ]:
graph(df)  # process 